In [2]:
! mkdir ImageClassification

In [3]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade --quiet {USER_FLAG} google-cloud-aiplatform kfp google-cloud-pipeline-components==1.0.40 google-cloud-storage


In [4]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


In [1]:
import os
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  qwiklabs-gcp-00-38ed10bd49be


In [2]:
! mkdir pipelines

In [3]:
REGION="us-central1"

In [4]:
import os

BUCKET=PROJECT_ID + "-bucket"

# set environment variable for reference later.
os.environ['REGION']=REGION
print(os.getenv('REGION'))

os.environ['BUCKET']=BUCKET
print(os.getenv('BUCKET'))

os.environ['PROJECT_ID']=PROJECT_ID
print(os.getenv('PROJECT_ID'))


us-central1
qwiklabs-gcp-00-38ed10bd49be-bucket
qwiklabs-gcp-00-38ed10bd49be


In [5]:
%%writefile requirements.txt
google-cloud-aiplatform
kfp
google-cloud-pipeline-components==1.0.40
google-cloud-storage

Writing requirements.txt


In [6]:
%%writefile pipelines/train_pipeline.py

import os

import kfp
import time
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics, Dataset

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from typing import NamedTuple, Dict

from google.cloud import aiplatform

#Main pipeline class
class pipeline_controller():
    def __init__(self, template_path, display_name, pipeline_root, project_id, region):
        self.template_path = template_path
        self.display_name = display_name
        self.pipeline_root = pipeline_root
        self.project_id = project_id
        self.region = region
    
    def _build_compile_pipeline(self):
        """Method to build and compile pipeline"""
        self.pipeline = self._get_pipeline(self.project_id, self.region)
        compiler.Compiler().compile(
            pipeline_func=self.pipeline, package_path=self.template_path
        )
        
    def _submit_job(self):
        """Method to Submit ML Pipeline job"""
        #Next, define the job:
        ml_pipeline_job = aiplatform.PipelineJob(
            display_name=self.display_name,
            template_path=self.template_path,
            pipeline_root=self.pipeline_root,
            project=self.project_id,
            location=self.region,
            # parameter_values={"project": self.project_id, "display_name": self.display_name},
            enable_caching=False
        )

        #And finally, run the job:
        ml_pipeline_job.submit()
    
    def _get_pipeline(self, PROJECT_ID, REGION):
        ## Light weight component to create an Image DS
        @component(
            base_image="python:3.9-slim",
            packages_to_install=["google-api-core==2.10.2", "google-cloud", "google-cloud-aiplatform", "typing", "kfp"],
        )
        def create_ds(project: str, 
                      display_name: str, 
                      gcs_source: str, 
                      import_schema_uri: str, 
                      timeout: int, 
                      dataset: Output[Dataset]):

            from google.cloud import aiplatform
            from google.cloud.aiplatform import datasets
            from kfp.v2.dsl import Dataset

            aiplatform.init(project=project)

            obj_dataset = datasets.ImageDataset.create(
                display_name=display_name,
                gcs_source=gcs_source,
                import_schema_uri=import_schema_uri,
                create_request_timeout=timeout,
            )

            obj_dataset.wait()

            dataset.uri = obj_dataset.gca_resource.name
            dataset.metadata = {
                'resourceName': obj_dataset.gca_resource.name
            }
        
        """Main method to Create pipeline"""
        @pipeline(name=self.display_name,
                    pipeline_root=self.pipeline_root)
        def pipeline_fn(
            project: str = PROJECT_ID, 
            region: str = REGION
        ):
            
            from google_cloud_pipeline_components import aiplatform as gcc_aip
            from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp, ModelDeployOp)
            import google.cloud.aiplatform as aip

            ds_op = create_ds(
                project=project,
                display_name="flowers",
                gcs_source="gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv",
                import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
                timeout=3600
            )

            training_job_run_op = gcc_aip.AutoMLImageTrainingJobRunOp(
                project=project,
                location=region,
                display_name="train-automl-flowers",
                prediction_type="classification",
                model_type="CLOUD",
                dataset=ds_op.outputs["dataset"].ignore_type(),
                model_display_name="train-automl-flowers",
                training_fraction_split=0.6,
                validation_fraction_split=0.2,
                test_fraction_split=0.2,
                budget_milli_node_hours=8000,
            )

            endpoint_op = EndpointCreateOp(
                project=project,
                location=region,
                display_name="train-automl-flowers",
            )

            ModelDeployOp(
                model=training_job_run_op.outputs["model"],
                endpoint=endpoint_op.outputs["endpoint"],
                automatic_resources_min_replica_count=1,
                automatic_resources_max_replica_count=1,
            )

        #Returns as the output of _get_pipeline()
        return pipeline_fn

Writing pipelines/train_pipeline.py


In [7]:
import os

BUCKET=PROJECT_ID + "-bucket"

# set environment variable for reference later.
os.environ['REGION']=REGION
print("REGION: " + os.getenv('REGION'))

os.environ['PROJECT_ID']=PROJECT_ID
print("PROJECT_ID: " + os.getenv('PROJECT_ID'))

os.environ['BUCKET']=BUCKET
print("BUCKET: " + os.getenv('BUCKET'))

REGION: us-central1
PROJECT_ID: qwiklabs-gcp-00-38ed10bd49be
BUCKET: qwiklabs-gcp-00-38ed10bd49be-bucket


In [8]:
%%writefile build_and_deploy.py

###Code to Build and Deploy the full pipeline
###This will be used in Cloud Builder

#Initialize pipeline object
from pipelines.train_pipeline import pipeline_controller
import time
import os

REGION="us-central1"
PROJECT_ID="qwiklabs-gcp-00-38ed10bd49be" 
BUCKET=f"{PROJECT_ID}-bucket"

PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root/"
DISPLAY_NAME = 'vertex-customml-pipeline{}'.format(str(int(time.time())))

print("Building pipeline {}".format(DISPLAY_NAME))

pipe = pipeline_controller(template_path="pipeline.json",
                           display_name="vertex-automlimage-classif", 
                           pipeline_root=PIPELINE_ROOT,
                           project_id=PROJECT_ID,
                           region=REGION)

#Build and Compile pipeline
pipe._build_compile_pipeline()

##Submit Job
pipe._submit_job()

Writing build_and_deploy.py


In [9]:
!python -m build_and_deploy

Building pipeline vertex-customml-pipeline1742223435
/home/jupyter/.local/lib/python3.10/site-packages/kfp/v2/compiler/compiler.py:1290: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(
Creating PipelineJob
PipelineJob created. Resource name: projects/26289415277/locations/us-central1/pipelineJobs/vertex-automlimage-classif-20250317145715
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/26289415277/locations/us-central1/pipelineJobs/vertex-automlimage-classif-20250317145715')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/vertex-automlimage-classif-20250317145715?project=26289415277


In [10]:
%%writefile Dockerfile

# Download dependencies to /root/.local.
FROM python:3.7-slim AS builder
COPY requirements.txt .

RUN pip install -r requirements.txt

COPY . .

Writing Dockerfile


In [11]:
!gcloud artifacts repositories create vertex-gar \
    --project=$PROJECT_ID \
    --repository-format=Docker \
    --location=$REGION


Create request issued for: [vertex-gar]
Waiting for operation [projects/qwiklabs-gcp-00-38ed10bd49be/locations/us-centr
al1/operations/68dc8cf1-4d73-4eaa-9c8d-31e69b345e60] to complete...done.       
Created repository [vertex-gar].


In [12]:
%%writefile cloudbuild.yaml

# Create config file and build a new image tagged with the given commit hash.

steps:
##Step 1 -> Build main image from Dockerfile
- name: 'gcr.io/cloud-builders/docker'
  id: 'build_image'
  args: [
    'build', '-t', '{REGION}-docker.pkg.dev/{PROJECT_ID}/vertex-gar/latest:latest',
    '-f', 'Dockerfile', '.',
  ]

##Step 2 -> Deploy Docker image to Artifact Repository
- name: 'gcr.io/cloud-builders/docker'
  id: 'push_image'
  waitFor:
    - 'build_image'
  args: [
    'push', '{REGION}-docker.pkg.dev/{PROJECT_ID}/vertex-gar/latest:latest'
  ]

##Step 3 -> Deploy pipeline to Vertex AI( using above built image )
- name: '{REGION}-docker.pkg.dev/{PROJECT_ID}/vertex-gar/latest:latest'
  id: 'deploy_pipelines'
  waitFor:
    - 'push_image'
  entrypoint: /bin/bash
  args:
    - -c
    - |
      python -m build_and_deploy

Writing cloudbuild.yaml


In [13]:
## Run the following command to replace {PROJECT_ID}/{REGION} with project id/region
cmd_str_proj = 's/[{]PROJECT_ID[}]/' + PROJECT_ID + '/g'
cmd_str_region = 's/[{]REGION[}]/' + REGION + '/g'
! sed -i $cmd_str_proj cloudbuild.yaml
! sed -i $cmd_str_region cloudbuild.yaml

In [14]:
!gcloud builds submit --config=cloudbuild.yaml

Creating temporary archive of 5908 file(s) totalling 160.6 MiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-00-38ed10bd49be_cloudbuild/source/1742224466.932253-d452608730804ce78704f25ea6c5fd7a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-38ed10bd49be/locations/global/builds/3a9c0177-c8aa-4f89-9130-d46d0c0276ce].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/3a9c0177-c8aa-4f89-9130-d46d0c0276ce?project=26289415277 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "3a9c0177-c8aa-4f89-9130-d46d0c0276ce"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-38ed10bd49be_cloudbuild/source/1742224466.932253-d452608730804ce78704f25ea6c5fd7a.tgz#1742224504520270
Copying gs://qwiklabs-gcp-00-38ed10bd49be_cloudbuild/source/1742224466.932253-d452608730804ce78704f25ea6c5fd7a.tgz#1742224504520270...
/ [